In [35]:
import pandas as pd

In [44]:
sub_data = pd.read_csv("foodstamp_submissions_allyears.csv")
com_data = pd.read_csv("foodstamp_comments_allyears.csv")

In [46]:
#removing rows with body having "moderator" word
com_data = com_data[~com_data['distinguished'].str.contains('moderator', case=False, na=False)]


In [48]:
# Group comments by link_id and concatenate all comments per post into a single string
grouped_comments = com_data.groupby('link_id')['body'].apply(' '.join).reset_index()

In [50]:
grouped_comments.columns
sub_data.columns

Index(['score', 'edited', 'quarantine', 'from_id', 'created', 'from_kind',
       'subreddit_id', 'url', 'secure_media', 'stickied', 'from', 'author',
       'is_self', 'num_comments', 'author_flair_css_class', 'over_18', 'saved',
       'name', 'gilded', 'link_flair_css_class', 'id', 'archived', 'selftext',
       'media', 'ups', 'retrieved_on', 'link_flair_text', 'permalink',
       'thumbnail', 'downs', 'distinguished', 'secure_media_embed', 'title',
       'author_flair_text', 'created_utc', 'media_embed', 'hide_score',
       'domain', 'subreddit', 'year_month'],
      dtype='object')

In [51]:
grouped_comments.columns

Index(['link_id', 'body'], dtype='object')

In [66]:
# Rename 'name' column to 'link_id' in df1 for consistency
sub_data.rename(columns={'name': 'link_id'}, inplace=True)

# Merge the DataFrames on 'link_id'
merged_df = pd.merge(sub_data, grouped_comments, on='link_id')

# Create the final DataFrame with selected columns
final_df = merged_df[['link_id','author','title', 'selftext', 'body','year_month']]

In [68]:

#Sorting the data on date for clarity
final_df['year_month'] = pd.to_datetime(final_df['year_month'], format='%Y-%m')
final_df = final_df.sort_values(by='year_month')

final_df.to_csv("merged.csv")

In [71]:
# Extract year and month for grouping
final_df['year'] = final_df['year_month'].dt.year
final_df['month'] = final_df['year_month'].dt.month

# Group by new year and month columns, then count rows
result = final_df.groupby(['year', 'month']).size().reset_index(name='counts')

# sorting the results
result = result.sort_values(by=['year', 'month'])

# Storing into csv
result.to_csv("Number_of_posts.csv")